In [90]:
import pandas as pd

In [91]:
dataset = pd.read_csv('GPUs.csv', skiprows=1, sep=';')

In [92]:
dataset.head()

,Unnamed: 0,Koszt,Wydajnosc,TDP,Ocena
0,Nvidia RTX 3060-Ti,2109,132.0,220,99
1,Nvidia RTX 3070,2383,154.0,220,99
2,Nvidia RTX 3050,1408,71.7,130,96
3,Nvidia GTX 1660S,1043,70.5,125,94
4,Nvidia RTX 3080,4266,206.0,320,91


In [93]:
dataset.describe()

,Koszt,Wydajnosc,TDP,Ocena
count,21.000000,21.000000,21.000000,21.000000
mean,4087.809524,167.376190,271.000000,70.000000
std,2451.496352,90.338192,89.145948,17.997222
min,1043.000000,70.500000,125.000000,50.000000
25%,2109.000000,96.200000,220.000000,57.000000
50%,3599.000000,147.000000,250.000000,60.000000
75%,6162.000000,206.000000,320.000000,90.000000
max,9699.000000,422.000000,450.000000,99.000000


# Promethe

In [94]:
# Preference funcions

def pref_v_shape(a, b, q, p):
    d = a - b
    if d > p:
        return 1
    elif d <= q:
        return 0
    else:
        return (d - q) / (p - q)

In [95]:
def comprehensive_preference_index(item1, item2, pref_info):
    val = 0.0
    for i in range(len(pref_info)):
        g = pref_info.iloc[i].name
        w = pref_info.iloc[i]["w"]
        q = pref_info.iloc[i]["q"]
        p = pref_info.iloc[i]["p"]
        a = item1[g]
        b = item2[g]
        pref = pref_info.iloc[i]["pref_func"](a, b, q, p)
        val += pref * w
    
    val /= pref_info["w"].sum()

    return val 

In [96]:
pref_info = pd.DataFrame({
    "g": ["Koszt", "Wydajnosc", "TDP", "Ocena"],
    "q": [100, 20, 10, 5],
    "p": [300, 50, 30, 30],
    "w": [3, 4, 1, 2],
    "pref_func": [pref_v_shape]*4
})

pref_info = pref_info.set_index("g")

In [97]:
pref_info

,q,p,w,pref_func
g,,,,
Koszt,100,300,3,<function pref_v_shape at 0x000001ABD73121F0>
Wydajnosc,20,50,4,<function pref_v_shape at 0x000001ABD73121F0>
TDP,10,30,1,<function pref_v_shape at 0x000001ABD73121F0>
Ocena,5,30,2,<function pref_v_shape at 0x000001ABD73121F0>


In [98]:
pref_info.iloc[0] 

q                                                      100
p                                                      300
w                                                        3
pref_func    <function pref_v_shape at 0x000001ABD73121F0>
Name: Koszt, dtype: object

In [99]:
pref_info.loc["Koszt"]

q                                                      100
p                                                      300
w                                                        3
pref_func    <function pref_v_shape at 0x000001ABD73121F0>
Name: Koszt, dtype: object

In [100]:
comprehensive_preference_index(dataset.iloc[0], dataset.iloc[1], pref_info)

0.0

In [103]:
result = dataset.apply(lambda row: dataset.apply(lambda col: comprehensive_preference_index(row[col.name], col[row.name], pref_info), axis=1))

TypeError: string indices must be integers